# Project: HVAC Performance Failure & Peak Temperature Event Detection

---

## Objective

To analyze temperature across monitoring sessions and identify abnormal peak temperature events that may indicate HVAC system issues or unusual environmental heat sources.

---

## Problem Statement

The objective is to query the historical sensor database to generate a comprehensive session-level temperature analysis, specifically:

- **Determining the typical operating temperature range** (minimum, maximum, and spread) for each monitoring session.
- **Identifying peak temperature events** where readings deviate significantly from session norms.
- **Correlating peak events with HVAC operation modes** to isolate instances where temperature anomalies occur despite active climate control.

---

## Impact

This analysis will be used to:

- **Establish baseline temperature profiles** for each monitoring session to enable comparative assessments.
- **Pinpoint sessions with HVAC performance degradation** where systems fail to maintain temperatures within acceptable ranges.
- **Detect unusual heat sources** contributing to temperature spikes unrelated to HVAC operation.
- **Prioritize maintenance interventions** by identifying sessions and conditions requiring immediate investigation.

---

## Definitions

| Term | Definition |
|------|------------|
| **Monitoring Session** | A distinct data collection period identified by a unique Session ID |
| **Operating Temperature Range** | The difference between maximum and minimum temperature readings within a session (Max - Min) |
| **Typical Operating Temperature** | The median temperature value within a session, representing central tendency |
| **Peak Temperature Event** | A temperature reading outside of acceptable range (10°C to 40°C) or deviating more than 3 standard deviations from the session median |
| **HVAC Mode** | System is in 'off' or 'maintenance_mode' status if no active climate control is activated |

---

## Scope

| Parameter | Value |
|-----------|-------|
| **Data Source** | Environmental sensor readings table |
| **Unit of Analysis** | Session ID |
| **Primary Metric** | Temperature |
| **Supporting Variables** | HVAC Operation Mode, Time of Day, Activity Level |

---

## Required Output Format

### Output 1: Session Temperature Summary

A summary table providing temperature metrics for each monitoring session.

**Example Output:**

| Session_ID | Reading_Count | Min_Temperature | Max_Temperature | Operating_Range | Median_Temperature | Avg_Temperature | Std_Deviation |
|------------|---------------|-----------------|-----------------|-----------------|--------------------|-----------------| --------------|
| 7241 | 4 | 19.63 | 20.10 | 0.47 | 19.78 | 19.82 | 0.18 |
| 9385 | 1 | 20.95 | 20.95 | 0.00 | 20.95 | 20.95 | 0.00 |

---
### Output 2: Peak Temperature Events

A detailed log of individual temperature readings classified as peak temperature events.

**Example Output:**

| Event_ID | Session_ID | Time_of_Day | Temperature | Session_Median | Deviation_From_Median | HVAC_Operation_Mode | Activity_Level | 
|----------|------------|-------------|-------------|----------------|----------------------|---------------------|----------------|
| 1 | 7241 | morning | 24.50 | 19.78 | 4.72 | off | High Activity | 
| 2 | 9385 | afternoon | 23.80 | 20.95 | 2.85 | eco_mode | Low Activity | 
---



### Output 3: Anomalous Sessions Alert List

A prioritized list of sessions requiring investigation or recommended to be dropped.

**Example Output:**

| Event_ID | Session_ID | Anomaly_Type | Peak_Temperature | CO_Level | CO2_Level | HVAC_Operation_Mode | Data_Handling_Action | Statistical_Reasoning |
|:---|:---|:---|:---|:---|:---|:---|:---|:---|
| 1 | 1374 | Signal Failure | 291.59°C | Normal | Normal | off | DROP | Unphysical value, signal fault. |
| 2 | 1388 | HVAC/Signal Failure | 291.59°C | Normal | Normal | cooling | INVESTIGATE | Unphysical value, hardware fault. |

---

## Success Criteria

| Criterion | Target |
|-----------|--------|
| All sessions analyzed | 100% coverage of valid Session IDs |
| Peak events identified | All readings outside acceptable range or above 3 std deviations flagged |
| HVAC correlation completed | Peak temperature events linked to HVAC status |
| Actionable output produced | Prioritized list of sessions requiring investigation |
| Output formats delivered | All 3 output tables generated with complete data |

### Loading the database


In [38]:
import sqlite3
import pandas as pd

# Connect to the SQLite db
conn = sqlite3.connect('./data/gas_monitoring.db') 

In [39]:
# Show the tables head for this database
df = pd.read_sql("SELECT * FROM gas_monitoring", conn)
display(df.head())

,Time of Day,Temperature,Humidity,CO2_InfraredSensor,CO2_ElectroChemicalSensor,MetalOxideSensor_Unit1,MetalOxideSensor_Unit2,MetalOxideSensor_Unit3,MetalOxideSensor_Unit4,CO_GasSensor,Session ID,HVAC Operation Mode,Ambient Light Level,Activity Level
0,morning,19.63,53.83,125.49,571.09,478.55,735.85,497.59,565.05,medium,7241,off,very_dim,Low Activity
1,morning,19.66,53.69,126.34,575.79,491.96,740.28,NaN,557.08,low,7241,ventilation_only,bright,Low Activity
2,night,20.95,54.12,119.34,556.53,437.50,718.34,NaN,612.62,medium,9385,off,None,Low Activity
3,morning,20.10,50.48,113.50,NaN,515.14,744.02,676.15,600.22,low,7241,eco_mode,very_bright,High Activity
4,morning,19.89,48.42,92.77,613.65,535.66,770.27,720.27,625.73,low,7241,Heating_active,moderate,Low Activity


## View Table

In [62]:
### Create a View for Common Used Table
# Remove the old view
conn.execute("DROP VIEW IF EXISTS median_per_session")

# Create the new view
query_view = """
CREATE VIEW median_per_session AS
  SELECT g1."Session ID" AS Session_ID, MIN(g1.Temperature) AS temperature_median
  FROM gas_monitoring g1
  WHERE (
    SELECT COUNT(*) 
    FROM gas_monitoring g2 
    WHERE g2."Session ID" = g1."Session ID" 
      AND g2.Temperature <= g1.Temperature
  ) * 1.0 / (SELECT COUNT(*) FROM gas_monitoring g3 WHERE g3."Session ID" = g1."Session ID") >= 0.5
  GROUP BY g1."Session ID"
"""
conn.execute(query_view)

### Output Table

### Output 1: Session Temperature Summary


In [63]:
# Run a query for the problem statement output 1.
# Method: Attempt in using correlated query (instead of window functions) to form the median of each session.
# Comment: SQLite does not contain MEDIAN() or STDDEV_POP() function.

# Remove the old view
conn.execute("DROP VIEW IF EXISTS readings_per_session")

# Create the new view
query_view1 = """

CREATE VIEW readings_per_session AS
  SELECT
    "Session ID" AS Session_ID,
    COUNT(*) AS Reading_Count,
    MIN(Temperature) AS Min_Temperature,
    MAX(Temperature) AS Max_Temperature,
    MAX(Temperature) - MIN(Temperature) AS Operating_Range,
    AVG(Temperature) AS Avg_Temperature,
    SQRT(AVG(Temperature * Temperature) - (AVG(Temperature) * AVG(Temperature))) AS Std_Deviation
  FROM gas_monitoring
  GROUP BY "Session ID"
  ORDER BY "Session ID"
"""
conn.execute(query_view1)

query1 = """
SELECT
  r.Session_ID,
  r.Reading_Count,
  r.Min_Temperature,
  r.Max_Temperature,
  r.Operating_Range,
  m.temperature_median AS Median_Temperature,
  r.Avg_Temperature,
  r.Std_Deviation
FROM readings_per_session r
JOIN median_per_session m
  ON r.Session_ID = m.Session_ID
GROUP BY r.Session_ID, m.temperature_median
ORDER BY r.Session_ID
"""
display(pd.read_sql(query1, conn, index_col='Session_ID'))

,Reading_Count,Min_Temperature,Max_Temperature,Operating_Range,Median_Temperature,Avg_Temperature,Std_Deviation
Session_ID,,,,,,,
1374,361,15.85,295.78,279.93,18.56,32.830222,59.476084
1458,165,18.95,296.82,277.87,21.49,39.554364,67.992640
1482,163,17.65,297.69,280.04,19.55,37.213558,65.856679
1826,282,18.01,298.53,280.52,21.98,43.759539,74.700453
1903,313,17.91,296.86,278.95,21.21,37.869872,65.294040
2105,337,17.07,296.74,279.67,19.24,42.436380,75.407526
2586,58,18.67,295.84,277.17,22.24,45.506724,76.694975
2679,274,14.94,296.50,281.56,19.04,40.005036,72.829467
2847,350,18.37,296.42,278.05,19.73,41.982886,74.063310


### Output Table

### Output 2: Peak Temperature Events


In [97]:
# Run a query for the problem statement output 2.
# Method: Using a Window function to generate an Event_ID in the SQL output and assign it as Index in Pandas.
# Comment:


# Remove the old view
conn.execute("DROP VIEW IF EXISTS events_by_session")

# Create the new view
query_view2 = """

CREATE VIEW events_by_session AS
SELECT 
  ROW_NUMBER() OVER (ORDER BY m.Session_ID) AS Event_ID, 
  m.Session_ID, 
  g."Time of Day" AS Time_Of_Day, 
  g.Temperature AS Peak_Temperature, 
  m.temperature_median AS Session_Median,
  ABS(g.Temperature - m.temperature_median) AS Deviation_From_Median,
  g."HVAC Operation Mode" AS HVAC_Operation_Mode,	
  g."Activity Level" AS Activity_Level,  
  g.CO2_ElectroChemicalSensor,
  g.CO_GasSensor
FROM gas_monitoring g
JOIN median_per_session m
  ON g."Session ID" = m.Session_ID 
WHERE 
g.Temperature < 10.0
OR g.Temperature > 40.0
OR ABS(g.Temperature - m.temperature_median) > 7.5
ORDER BY Event_ID

"""

conn.execute(query_view2)

query2 = """
SELECT 
  Event_ID, 
  Session_ID, 
  Time_Of_Day, 
  Peak_Temperature, 
  Session_Median,
  Deviation_From_Median,
  HVAC_Operation_Mode,	
  Activity_Level
FROM events_by_session

"""

display(pd.read_sql(query2, conn, index_col='Event_ID'))

,Session_ID,Time_Of_Day,Peak_Temperature,Session_Median,Deviation_From_Median,HVAC_Operation_Mode,Activity_Level
Event_ID,,,,,,,
1,1374,night,291.59,18.56,273.03,maintenance_mode,Low Activity
2,1374,morning,289.15,18.56,270.59,ventilation_only,Low Activity
3,1374,morning,289.41,18.56,270.85,eco_mode,Moderate Activity
4,1374,afternoon,290.54,18.56,271.98,maintenance_mode,Moderate Activity
5,1374,evening,295.78,18.56,277.22,heating_active,Moderate Activity
...,...,...,...,...,...,...,...
801,9658,afternoon,291.38,19.63,271.75,cooling_active,Moderate Activity
802,9658,morning,291.02,19.63,271.39,cooling_active,Low Activity
803,9658,night,291.28,19.63,271.65,off,High Activity


### Output Table

### Output 3: Anomalous Sessions Alert List


In [111]:
# Run a query for the problem statement output 3.
# METHODOLOGY: 
# Using Sensor Fusion (Temp + CO) and HVAC State to determine data validity.
#
# 1. OUTLIER TO DROP
#    Condition: HVAC 'Off'/'Maintenance' + Normal CO2 & CO level + Extreme Temperature
#    Action:    Signal Failure -> DROP (Instrumentation error).
#
# 2. OUTLIER TO INVESTIGATE
#    Condition: HVAC On + Normal CO2 & CO level + Extreme Temperature
#    Condition: HVAC On + Elevated CO2 OR CO level + Extreme Temperature
#    Action:    Possible HVAC/Sensor Failure -> KEEP & FLAG (Mechanical/Sensor check).
#
# 3. POSSIBLE PEAK TEMPERATURE EVENT
#    Condition: HVAC On + Elevated CO2 OR CO level + Peak (Not Extreme Temperature)
#    Action:    Possible Correlation -> KEEP & FLAG (Potential Fire/Thermal Event).
#
# 4. POSSIBLE HVAC INFLUENCED EVENT
#    Condition: HVAC Heating + Normal CO2 level + Peak Temperature
#    Condition: HVAC Cooling + Normal CO2 level + Peak Temperature
#    Action:    Normal Condition -> KEEP (Expected Operational Variance).
# =============================================================================

# COMMENT: This is a data cleaning approach to ensure unphysical hardware errors (Signal Failures) 
# are removed while protecting credible physical and mechanical anomalies for analysis.

query3 = """
WITH filtered_events AS (
    SELECT 
        Event_ID, 
        Session_ID, 
        Peak_Temperature, 
        CO2_ElectroChemicalSensor,
        LOWER(CO_GasSensor) AS CO_Gas_sensor, 
        LOWER(HVAC_Operation_Mode) AS HVAC_Operation_mode
    FROM events_by_session
    WHERE Peak_Temperature > 250
),
cat_events AS (
    SELECT 
        *,
        CASE
            WHEN CO2_ElectroChemicalSensor <= 1500 
                 AND CO_Gas_sensor LIKE '%low%' 
                 AND HVAC_Operation_mode IN ('off', 'maintenance_mode')
            THEN 'DROP'
            WHEN HVAC_Operation_mode NOT IN ('off', 'maintenance_mode')
                 AND ((CO2_ElectroChemicalSensor <= 1500 AND CO_Gas_sensor LIKE '%low%') 
                      OR (CO2_ElectroChemicalSensor > 1500 
                          AND CO_Gas_sensor NOT LIKE '%low%'
                          AND CO_Gas_sensor NOT LIKE '%none%'))
            THEN 'INVESTIGATE' 
        END AS Data_Handling_Action
    FROM filtered_events
)
SELECT 
    Event_ID, 
    Session_ID, 
    CASE Data_Handling_Action
        WHEN 'DROP' 
            THEN 'Signal Fault'
        WHEN 'INVESTIGATE' 
            THEN 'HVAC Fault'
    END AS Anomaly_Type,
    Peak_Temperature, 
    CO2_ElectroChemicalSensor,
    CO_Gas_sensor, 
    HVAC_Operation_mode,
    CASE Data_Handling_Action
        WHEN 'DROP' 
            THEN 'Unphysical value, signal fault'
        WHEN 'INVESTIGATE' 
            THEN 'Unphysical value, HVAC failure'
    END AS Comments
FROM cat_events
WHERE Data_Handling_Action IS NOT NULL
ORDER BY Event_ID
"""

display(pd.read_sql(query3, conn, index_col='Event_ID'))

,Session_ID,Anomaly_Type,Peak_Temperature,CO2_ElectroChemicalSensor,CO_Gas_sensor,HVAC_Operation_mode,Comments
Event_ID,,,,,,,
1,1374,Signal Fault,291.59,590.50,low,maintenance_mode,"Unphysical value, signal fault"
2,1374,HVAC Fault,289.15,622.37,extremely low,ventilation_only,"Unphysical value, HVAC failure"
3,1374,HVAC Fault,289.41,615.41,extremely low,eco_mode,"Unphysical value, HVAC failure"
5,1374,HVAC Fault,295.78,583.77,low,heating_active,"Unphysical value, HVAC failure"
9,1374,HVAC Fault,291.87,591.31,low,heating_active,"Unphysical value, HVAC failure"
...,...,...,...,...,...,...,...
796,9658,HVAC Fault,291.13,586.12,low,heating_active,"Unphysical value, HVAC failure"
797,9658,Signal Fault,292.84,586.00,low,maintenance_mode,"Unphysical value, signal fault"
800,9658,HVAC Fault,291.83,598.02,low,ventilation_only,"Unphysical value, HVAC failure"


### End the database connection

In [ ]:
# Close connection to db
conn.close()
print("Successfully closed connection to SQLite")